This notebook best viewed here: https://nbviewer.jupyter.org

First I use LD-pruned SNPs (from 001_JP_pooled_pangenome_data_explore.ipynb) to estimate the covariance matrix across 5 independent runs of the core mode of baypass, and to ensure convergence I calculate the pairwise correlation of the matrices across runs (I found that these runs were highly correlated, r2 > 0.998).

Then I run baypass in STD mode (mcmc) for SNPs in our data.

In [10]:
from pythonimports import *

DIR = '/data/projects/pool_seq/pangenome/JP_pangenome'
snpdir = op.join(DIR, 'JP_pooled/snpsANDindels/03_maf-p05_RD-recalculated')
baydir = makedir(op.join(snpdir, 'baypass'))
op.exists(baydir)

True

# create infiles to baypass to estimate neutral structure

#### <center> genotyping data

"The genotyping data file is simply organized as a matrix with
nsnp rows and 2 ∗ npop columns. The row field separator is a space. More
precisely, each row corresponds to one marker and the number of columns is
twice the number of populations because each pair of numbers corresponds to
each allele (or read counts for PoolSeq experiment) counts in one population"

In [2]:
ls(baydir)

['no-missing-data_20-dp-1000_random-snps_1-per-contig-gt100Kbp_r2-lessthan-p36068.pkl',
 'no-missing-data_20-dp-1000_random-snps_1-per-contig-gt100Kbp_r2-lessthan-p36068.txt']

In [3]:
# load snps that are LD-pruned
# these loci were chosen in 001_JP_pooled_pangenome_data_explore.ipynb
prunedloci = pklload(op.join(baydir, 'no-missing-data_20-dp-1000_random-snps_1-per-contig-gt100Kbp_r2-lessthan-p36068.pkl'))
len(prunedloci)

8591

In [12]:
# load snp data, reduce to only the randomloci
snps = pd.read_table(op.join(snpdir, 'JP_pooled-varscan_all_bedfiles_SNP_maf_RD-recalculated.txt'))
reduced = snps[snps['unstitched_locus'].isin(prunedloci)]
reduced.index = reduced['unstitched_locus'].tolist()  # adding locus name to the index will help look up AD/RD info for a locus
print(reduced.shape, snps.shape)  # want to make sure rows of reduced == len(randomloci)
reduced.head()

(8591, 338) (1235752, 338)


,CHROM,POS,REF,ALT,AF,QUAL,TYPE,FILTER,ADP,WT,HET,HOM,NC,JP_p98.GT,JP_p98.GQ,JP_p98.SDP,JP_p98.DP,JP_p98.FREQ,JP_p98.PVAL,JP_p98.AD,JP_p98.RD,JP_p31.GT,JP_p31.GQ,JP_p31.SDP,JP_p31.DP,JP_p31.FREQ,JP_p31.PVAL,JP_p31.AD,JP_p31.RD,JP_p93.GT,JP_p93.GQ,JP_p93.SDP,JP_p93.DP,JP_p93.FREQ,JP_p93.PVAL,JP_p93.AD,JP_p93.RD,JP_p24.GT,JP_p24.GQ,JP_p24.SDP,JP_p24.DP,JP_p24.FREQ,JP_p24.PVAL,JP_p24.AD,JP_p24.RD,JP_p14.GT,JP_p14.GQ,JP_p14.SDP,JP_p14.DP,JP_p14.FREQ,...,JP_p102.DP,JP_p102.FREQ,JP_p102.PVAL,JP_p102.AD,JP_p102.RD,JP_p100.GT,JP_p100.GQ,JP_p100.SDP,JP_p100.DP,JP_p100.FREQ,JP_p100.PVAL,JP_p100.AD,JP_p100.RD,JP_p42.GT,JP_p42.GQ,JP_p42.SDP,JP_p42.DP,JP_p42.FREQ,JP_p42.PVAL,JP_p42.AD,JP_p42.RD,JP_p103.GT,JP_p103.GQ,JP_p103.SDP,JP_p103.DP,JP_p103.FREQ,JP_p103.PVAL,JP_p103.AD,JP_p103.RD,JP_p48.GT,JP_p48.GQ,JP_p48.SDP,JP_p48.DP,JP_p48.FREQ,JP_p48.PVAL,JP_p48.AD,JP_p48.RD,JP_p79.GT,JP_p79.GQ,JP_p79.SDP,JP_p79.DP,JP_p79.FREQ,JP_p79.PVAL,JP_p79.AD,JP_p79.RD,locus,unstitched_chrom,unstitched_pos,unstitched_locus,MAF
>super4-47993,Scaffold_1,47993,C,G,0.051569,-10.0,SNP,PASS,66,23,17,0,0,C/C,60.0,44,44.0,4.55%,0.247130,2.0,42.0,C/C,119.0,63,63.0,0%,1.000000,0.0,63.0,C/C,67.0,35,35.0,0%,1.000000e+00,0.0,35.0,C/C,167.0,90,90.0,0%,1.000000e+00,0.0,90.0,C/G,21.0,88,88.0,7.95%,...,26.0,0%,1.000000,0.0,26.0,C/C,74.0,39,39.0,0%,1.000000e+00,0.0,39.0,C/C,142.0,75,75.0,0%,1.000000,0.0,75.0,C/C,89.0,47,47.0,0%,1.000000,0.0,47.0,C/G,31.0,98,98.0,10.2%,0.000767,10.0,88.0,C/G,38.0,67,67.0,17.91%,0.000142,12.0,55.0,Scaffold_1-47993,>super4,47993,>super4-47993,0.051569
>super124-1063998,Scaffold_4,1063998,G,T,0.199180,-10.0,SNP,PASS,65,5,35,0,0,G/T,25.0,58,58.0,13.79%,0.003017,8.0,50.0,G/G,44.0,33,33.0,6.06%,0.246150,2.0,31.0,G/T,63.0,65,65.0,29.23%,4.086100e-07,19.0,46.0,G/T,95.0,54,54.0,48.15%,2.717100e-10,26.0,28.0,G/G,95.0,73,73.0,5.48%,...,33.0,27.27%,0.001042,9.0,24.0,G/T,120.0,71,71.0,46.48%,8.710600e-13,33.0,38.0,G/T,31.0,70,70.0,14.29%,0.000692,10.0,60.0,G/T,15.0,56,56.0,8.93%,0.028473,5.0,51.0,G/T,48.0,74,74.0,20.27%,0.000014,15.0,59.0,G/T,15.0,83,83.0,6.02%,0.029373,5.0,78.0,Scaffold_4-1063998,>super124,1063998,>super124-1063998,0.199180
>super131-87067,Scaffold_5,87067,A,G,0.178779,-10.0,SNP,PASS,68,5,35,0,0,A/G,45.0,68,68.0,20.59%,0.000029,14.0,54.0,A/G,33.0,38,38.0,26.32%,0.000495,10.0,28.0,A/G,21.0,76,76.0,9.21%,6.762200e-03,7.0,69.0,A/A,149.0,79,79.0,0%,1.000000e+00,0.0,79.0,A/G,48.0,84,84.0,17.86%,...,40.0,12.5%,0.027371,5.0,35.0,A/G,28.0,46,46.0,19.57%,1.268200e-03,9.0,37.0,A/G,31.0,66,66.0,15.15%,0.000676,10.0,56.0,A/G,31.0,77,77.0,12.99%,0.000715,10.0,67.0,A/G,41.0,71,71.0,18.31%,0.000067,13.0,58.0,A/G,18.0,66,66.0,9.09%,0.013877,6.0,60.0,Scaffold_5-87067,>super131,87067,>super131-87067,0.178779
>super144-161322,Scaffold_6,161322,A,T,0.065424,-10.0,SNP,PASS,49,28,12,0,0,A/A,71.0,49,49.0,4.08%,0.247420,2.0,47.0,A/A,73.0,45,45.0,2.22%,0.500000,1.0,44.0,A/A,130.0,70,70.0,0%,1.000000e+00,0.0,70.0,A/T,22.0,44,44.0,15.91%,6.036300e-03,7.0,37.0,A/T,18.0,62,62.0,9.68%,...,21.0,9.52%,0.243900,2.0,19.0,A/T,22.0,40,40.0,17.5%,5.868800e-03,7.0,33.0,A/A,57.0,41,41.0,4.88%,0.246910,2.0,39.0,A/A,95.0,63,63.0,3.17%,0.248000,2.0,61.0,A/A,57.0,42,42.0,4.76%,0.246990,2.0,40.0,A/T,21.0,53,53.0,13.21%,0.006325,7.0,46.0,Scaffold_6-161322,>super144,161322,>super144-161322,0.065424
>super215-1038994,Scaffold_7,1038994,G,A,0.072096,-10.0,SNP,PASS,236,0,40,0,0,G/A,36.0,195,195.0,6.15%,0.000205,12.0,183.0,G/A,18.0,223,223.0,2.69%,0.015100,6.0,217.0,G/A,62.0,234,234.0,8.55%,6.242500e-07,20.0,214.0,G/A,58.0,315,315.0,6.03%,1.442000e-06,19.0,296.0,G/A,64.0,280,280.0,7.5%,...,107.0,6.54%,0.007060,7.0,100.0,G/A,36.0,172,172.0,6.98%,2.001800e-04,12.0,160.0,G/A,46.0,256,256.0,5.86%,0.000025,15.0,241.0,G/A,49.0,209,209.0,7.66%,0.000011,16.0,193.0,G/A,42.0,270,270.0,5.19%,0.000051,14.0,256.0,G/A,55.0,268,268.0,6.72%,0.000003,18.0,250.0,Scaffold_7-1038994,>super215,1038994,>super215-1038994,0.072096


In [13]:
snps.head()

,CHROM,POS,REF,ALT,AF,QUAL,TYPE,FILTER,ADP,WT,HET,HOM,NC,JP_p98.GT,JP_p98.GQ,JP_p98.SDP,JP_p98.DP,JP_p98.FREQ,JP_p98.PVAL,JP_p98.AD,JP_p98.RD,JP_p31.GT,JP_p31.GQ,JP_p31.SDP,JP_p31.DP,JP_p31.FREQ,JP_p31.PVAL,JP_p31.AD,JP_p31.RD,JP_p93.GT,JP_p93.GQ,JP_p93.SDP,JP_p93.DP,JP_p93.FREQ,JP_p93.PVAL,JP_p93.AD,JP_p93.RD,JP_p24.GT,JP_p24.GQ,JP_p24.SDP,JP_p24.DP,JP_p24.FREQ,JP_p24.PVAL,JP_p24.AD,JP_p24.RD,JP_p14.GT,JP_p14.GQ,JP_p14.SDP,JP_p14.DP,JP_p14.FREQ,...,JP_p102.DP,JP_p102.FREQ,JP_p102.PVAL,JP_p102.AD,JP_p102.RD,JP_p100.GT,JP_p100.GQ,JP_p100.SDP,JP_p100.DP,JP_p100.FREQ,JP_p100.PVAL,JP_p100.AD,JP_p100.RD,JP_p42.GT,JP_p42.GQ,JP_p42.SDP,JP_p42.DP,JP_p42.FREQ,JP_p42.PVAL,JP_p42.AD,JP_p42.RD,JP_p103.GT,JP_p103.GQ,JP_p103.SDP,JP_p103.DP,JP_p103.FREQ,JP_p103.PVAL,JP_p103.AD,JP_p103.RD,JP_p48.GT,JP_p48.GQ,JP_p48.SDP,JP_p48.DP,JP_p48.FREQ,JP_p48.PVAL,JP_p48.AD,JP_p48.RD,JP_p79.GT,JP_p79.GQ,JP_p79.SDP,JP_p79.DP,JP_p79.FREQ,JP_p79.PVAL,JP_p79.AD,JP_p79.RD,locus,unstitched_chrom,unstitched_pos,unstitched_locus,MAF
0,Scaffold_1,15421,A,G,0.674211,-10.0,SNP,PASS,22,0,32,8,0,A/G,67.0,22,22.0,72.73%,1.790500e-07,16.0,6.0,G/G,126.0,32,32.0,84.38%,2.378500e-13,27.0,5.0,A/G,87.0,26,26.0,76.92%,1.827300e-09,20.0,6.0,G/G,113.0,28,28.0,85.71%,4.701500e-12,24.0,4.0,A/G,30.0,14,14.0,57.14%,...,15.0,80%,5.260500e-06,12.0,3.0,A/G,21.0,12,12.0,50%,6.865000e-03,6.0,6.0,A/G,97.0,28,28.0,78.57%,1.758300e-10,22.0,6.0,G/G,49.0,13,13.0,84.62%,1.009600e-05,11.0,2.0,A/G,38.0,26,26.0,42.31%,1.279100e-04,11.0,15.0,A/G,47.0,19,19.0,63.16%,1.861100e-05,12.0,7.0,Scaffold_1-15421,>super4,15421,>super4-15421,0.325789
1,Scaffold_1,47333,G,A,0.105707,-10.0,SNP,PASS,109,7,33,0,0,G/A,15.0,61,61.0,8.2%,2.870000e-02,5.0,56.0,G/A,34.0,121,121.0,9.09%,3.850000e-04,11.0,110.0,G/A,58.0,85,85.0,21.18%,1.395700e-06,18.0,67.0,G/G,268.0,144,144.0,0%,1.000000e+00,0.0,144.0,G/A,34.0,133,133.0,8.27%,...,44.0,18.18%,2.757300e-03,8.0,36.0,G/A,21.0,77,77.0,9.09%,6.775500e-03,7.0,70.0,G/A,18.0,109,109.0,5.5%,1.456000e-02,6.0,103.0,G/A,44.0,103,103.0,13.59%,3.802900e-05,14.0,89.0,G/G,250.0,134,134.0,0%,1.000000e+00,0.0,134.0,G/A,27.0,105,105.0,8.57%,1.633500e-03,9.0,96.0,Scaffold_1-47333,>super4,47333,>super4-47333,0.105707
2,Scaffold_1,47418,T,A,0.217593,-10.0,SNP,PASS,113,2,38,0,0,T/A,45.0,70,70.0,20%,2.968700e-05,14.0,56.0,T/A,83.0,140,140.0,18.57%,4.146600e-09,26.0,114.0,T/A,82.0,97,97.0,25.77%,5.045400e-09,25.0,72.0,T/A,33.0,155,155.0,7.1%,4.063500e-04,11.0,144.0,T/A,94.0,137,137.0,21.17%,...,44.0,15.91%,6.036300e-03,7.0,37.0,T/A,51.0,89,89.0,17.98%,7.216200e-06,16.0,73.0,T/A,71.0,112,112.0,19.64%,7.603100e-08,22.0,90.0,T/A,71.0,102,102.0,21.57%,6.704300e-08,22.0,80.0,T/A,37.0,121,121.0,9.92%,1.833300e-04,12.0,109.0,T/A,67.0,122,122.0,17.21%,1.861000e-07,21.0,101.0,Scaffold_1-47418,>super4,47418,>super4-47418,0.217593
3,Scaffold_1,47461,T,G,0.090661,-10.0,SNP,PASS,109,12,28,0,0,T/T,145.0,77,77.0,0%,1.000000e+00,0.0,77.0,T/T,250.0,133,133.0,0%,1.000000e+00,0.0,133.0,T/G,15.0,103,103.0,4.85%,2.973700e-02,5.0,98.0,T/G,235.0,161,161.0,40.99%,2.881900e-24,66.0,95.0,T/G,27.0,120,120.0,7.5%,...,46.0,0%,1.000000e+00,0.0,46.0,T/G,31.0,84,84.0,11.9%,7.350700e-04,10.0,74.0,T/T,162.0,102,102.0,1.96%,2.487700e-01,2.0,100.0,T/T,144.0,91,91.0,2.2%,2.486200e-01,2.0,89.0,T/G,77.0,123,123.0,19.51%,1.720100e-08,24.0,99.0,T/G,77.0,125,125.0,19.2%,1.758400e-08,24.0,101.0,Scaffold_1-47461,>super4,47461,>super4-47461,0.090661
4,Scaffold_1,47513,C,G,0.520739,-10.0,SNP,PASS,112,0,40,0,0,C/G,165.0,77,77.0,55.84%,2.805900e-17,43.0,34.0,C/G,255.0,133,133.0,56.39%,9.126200e-30,75.0,58.0,C/G,134.0,97,97.0,39.18%,3.898700e-14,38.0,59.0,C/G,62.0,168,168.0,11.9%,5.230200e-07,20.0,148.0,C/G,179.0,96,96.0,50%,...,46.0,63.04%,2.465900e-12,29.0,17.0,C/G,143.0,83,83.0,46.99%,4.966300e-15,39.0,44.0,C/G,211.0,116,116.0,49.14%,6.616500e-22,57.0,59.0,C/G,192.0,90,90.0,55.56%,5.861500e-20,50.0,40.0,C/G,255.0,139,139.0,69.78%,1.159100e-41,97.0,42.0,C/G,219.0,120,120.0,49.17%,1.178100e-22,59.0,61.0,Scaffold_1-47513,>super4,47513,>sup

In [14]:
def get_counts(loci):
    """Get read counts for global major and minor allele."""
    import pandas
    from collections import OrderedDict
    import tqdm
    
    pops = [col.replace(".FREQ","") for col in reduced.columns if '.FREQ' in col]
    assert len(pops) == 40
    read_counts = OrderedDict()
    for locus in tqdm.tqdm_notebook(loci):
        if reduced.loc[locus, 'AF'] > 0.5:
            # this matches calculation of MAF from 001_JP_pooled_pangenome_data_explore.ipynb
            refismajor = False
            majortag = 'AD'
            minortag = 'RD'
        else:
            refismajor = True
            majortag = 'RD'
            minortag = 'AD'
        if refismajor is True:
#             print('checking ', locus)
            # double check that it matches MAF calculation
            assert reduced.loc[locus, 'AF'] == reduced.loc[locus, 'MAF']
        for pop in pops:
            for which,tag in zip(['major','minor'],[majortag, minortag]):
                newcol = "%s-%s" % (pop,which)
                if newcol not in read_counts:
                    read_counts[newcol] = OrderedDict()
                try:
                    read_counts[newcol][locus] = int(reduced.loc[locus, "%s.%s" % (pop, tag)])
                except ValueError as e:
                    # missing data
                    read_counts[newcol][locus] = 0
    return pandas.DataFrame(read_counts)

In [15]:
# get the gfile using the prunedloci
neutral_read_counts = get_counts(prunedloci)
neutral_read_counts.head()

/data/programs/brandon_anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


,JP_p98-major,JP_p98-minor,JP_p31-major,JP_p31-minor,JP_p93-major,JP_p93-minor,JP_p24-major,JP_p24-minor,JP_p14-major,JP_p14-minor,JP_p70-major,JP_p70-minor,JP_p8-major,JP_p8-minor,JP_p95-major,JP_p95-minor,JP_p57-major,JP_p57-minor,JP_p96-major,JP_p96-minor,JP_p51-major,JP_p51-minor,JP_p73-major,JP_p73-minor,JP_p83-major,JP_p83-minor,JP_p26-major,JP_p26-minor,JP_p68-major,JP_p68-minor,JP_p6-major,JP_p6-minor,JP_p92-major,JP_p92-minor,JP_p72-major,JP_p72-minor,JP_p94-major,JP_p94-minor,JP_p62-major,JP_p62-minor,JP_p99-major,JP_p99-minor,JP_p39-major,JP_p39-minor,JP_p97-major,JP_p97-minor,JP_p90-major,JP_p90-minor,JP_p37-major,JP_p37-minor,JP_p84-major,JP_p84-minor,JP_p27-major,JP_p27-minor,JP_p82-major,JP_p82-minor,JP_p29-major,JP_p29-minor,JP_p20-major,JP_p20-minor,JP_p77-major,JP_p77-minor,JP_p4-major,JP_p4-minor,JP_p101-major,JP_p101-minor,JP_p60-major,JP_p60-minor,JP_p102-major,JP_p102-minor,JP_p100-major,JP_p100-minor,JP_p42-major,JP_p42-minor,JP_p103-major,JP_p103-minor,JP_p48-major,JP_p48-minor,JP_p79-major,JP_p79-minor
>scaffold125916-52766,47,20,32,8,53,16,65,22,45,27,63,10,18,23,69,5,52,20,54,16,70,12,46,20,56,3,67,26,70,2,38,22,79,22,82,11,57,33,56,9,59,7,66,16,57,10,44,9,34,11,62,18,35,34,16,10,46,26,30,26,74,5,17,13,26,12,74,31,13,13,44,13,35,30,50,19,58,12,72,16
>scaffold48955-25539,51,11,66,2,84,5,90,0,58,22,67,8,76,1,59,6,76,3,71,11,82,14,70,5,73,3,115,0,82,25,50,13,100,8,77,7,81,8,76,3,79,0,95,0,77,3,73,0,56,4,88,8,81,24,55,1,80,13,65,6,87,15,50,1,43,1,50,11,49,1,73,5,81,7,86,7,88,5,94,0
>scaffold57006-68497,110,8,121,20,145,24,168,0,170,0,113,22,83,7,123,25,140,17,130,33,162,15,133,21,110,19,149,26,129,17,63,33,180,9,183,19,149,18,115,22,116,6,129,32,98,36,63,11,85,8,119,21,156,7,78,5,123,33,80,49,106,23,74,3,90,6,153,7,65,7,91,15,130,17,88,30,106,52,144,12
>super297-68268,168,28,117,16,194,20,210,19,195,22,227,36,118,15,263,28,219,24,220,33,177,25,156,18,217,22,243,43,214,20,155,30,363,53,222,19,322,35,251,23,218,26,203,24,224,15,180,18,133,18,273,19,169,21,92,18,229,24,185,24,208,19,107,16,136,14,193,34,119,11,194,25,198,23,221,41,247,17,187,22
>super2840-258793,56,7,45,0,60,4,65,0,64,1,70,15,33,1,57,3,59,2,65,5,74,3,63,16,50,8,101,2,68,7,42,7,111,3,60,12,68,1,74,6,63,2,76,2,70,6,59,2,47,1,76,4,69,0,32,5,63,4,49,6,70,2,36,2,31,2,58,5,38,1,58,2,60,7,59,2,52,5,64,0


In [21]:
for locus in neutral_read_counts.index:
    ad = reduced.loc[locus, 'JP_p98.AD']
    rd = reduced.loc[locus, 'JP_p98.RD']
    dp = reduced.loc[locus, 'JP_p98.DP']
    assert ad+rd==dp

In [22]:
baydir

'/data/projects/pool_seq/pangenome/JP_pangenome/JP_pooled/snpsANDindels/03_maf-p05_RD-recalculated/baypass'

In [23]:
# save
neutral_read_counts.to_csv(op.join(baydir, 'neutral_gfile_HEADERIDX.txt'), sep='\t', index=True)
neutral_read_counts.to_csv(op.join(baydir, 'neutral_gfile_noheaderidx.txt'), sep='\t', index=False, header=False)

#### <center> poolsizefile

"For Pool–Seq experiment, the haploid size (twice the number of pooled in- dividuals for diploid species) of each population should be provided. ... The order of the populations in the pool size file must be the same as in the allele count (and the covariate) data file(s)."

In [29]:
# the reason the ploidy.keys() are not in same order of table ...
# ... is because varscan puts Sample1.X then Sample10.X ... (JP_p31 is the 10th file into varscan)
ploidy = pklload(op.join(DIR, 'JP_pooled/pkl_files/ploidy.pkl'))['JP_pooled']
count = 0
for k,v in ploidy.items():
    print(k,v)
    count += 1
    if count == 5:
        break

JP_p98 40
JP_p73 40
JP_p97 40
JP_p60 40
JP_p100 40


In [30]:
# get ploidy of pops in same order as in gfile
pops = [col.replace("-major","") for col in neutral_read_counts.columns if '-major' in col]
vals = OrderedDict()
for pop in pops:
    vals[pop] = OrderedDict()
    vals[pop][0] = ploidy[pop]
poolsizefile = pd.DataFrame(vals)
poolsizefile

,JP_p98,JP_p31,JP_p93,JP_p24,JP_p14,JP_p70,JP_p8,JP_p95,JP_p57,JP_p96,JP_p51,JP_p73,JP_p83,JP_p26,JP_p68,JP_p6,JP_p92,JP_p72,JP_p94,JP_p62,JP_p99,JP_p39,JP_p97,JP_p90,JP_p37,JP_p84,JP_p27,JP_p82,JP_p29,JP_p20,JP_p77,JP_p4,JP_p101,JP_p60,JP_p102,JP_p100,JP_p42,JP_p103,JP_p48,JP_p79
0,40,40,40,34,40,40,40,40,40,40,40,40,38,40,40,40,40,40,40,38,40,40,40,38,40,40,40,38,40,40,40,40,40,40,40,40,40,40,40,40


In [31]:
neutral_read_counts.head()

,JP_p98-major,JP_p98-minor,JP_p31-major,JP_p31-minor,JP_p93-major,JP_p93-minor,JP_p24-major,JP_p24-minor,JP_p14-major,JP_p14-minor,JP_p70-major,JP_p70-minor,JP_p8-major,JP_p8-minor,JP_p95-major,JP_p95-minor,JP_p57-major,JP_p57-minor,JP_p96-major,JP_p96-minor,JP_p51-major,JP_p51-minor,JP_p73-major,JP_p73-minor,JP_p83-major,JP_p83-minor,JP_p26-major,JP_p26-minor,JP_p68-major,JP_p68-minor,JP_p6-major,JP_p6-minor,JP_p92-major,JP_p92-minor,JP_p72-major,JP_p72-minor,JP_p94-major,JP_p94-minor,JP_p62-major,JP_p62-minor,JP_p99-major,JP_p99-minor,JP_p39-major,JP_p39-minor,JP_p97-major,JP_p97-minor,JP_p90-major,JP_p90-minor,JP_p37-major,JP_p37-minor,JP_p84-major,JP_p84-minor,JP_p27-major,JP_p27-minor,JP_p82-major,JP_p82-minor,JP_p29-major,JP_p29-minor,JP_p20-major,JP_p20-minor,JP_p77-major,JP_p77-minor,JP_p4-major,JP_p4-minor,JP_p101-major,JP_p101-minor,JP_p60-major,JP_p60-minor,JP_p102-major,JP_p102-minor,JP_p100-major,JP_p100-minor,JP_p42-major,JP_p42-minor,JP_p103-major,JP_p103-minor,JP_p48-major,JP_p48-minor,JP_p79-major,JP_p79-minor
>scaffold125916-52766,47,20,32,8,53,16,65,22,45,27,63,10,18,23,69,5,52,20,54,16,70,12,46,20,56,3,67,26,70,2,38,22,79,22,82,11,57,33,56,9,59,7,66,16,57,10,44,9,34,11,62,18,35,34,16,10,46,26,30,26,74,5,17,13,26,12,74,31,13,13,44,13,35,30,50,19,58,12,72,16
>scaffold48955-25539,51,11,66,2,84,5,90,0,58,22,67,8,76,1,59,6,76,3,71,11,82,14,70,5,73,3,115,0,82,25,50,13,100,8,77,7,81,8,76,3,79,0,95,0,77,3,73,0,56,4,88,8,81,24,55,1,80,13,65,6,87,15,50,1,43,1,50,11,49,1,73,5,81,7,86,7,88,5,94,0
>scaffold57006-68497,110,8,121,20,145,24,168,0,170,0,113,22,83,7,123,25,140,17,130,33,162,15,133,21,110,19,149,26,129,17,63,33,180,9,183,19,149,18,115,22,116,6,129,32,98,36,63,11,85,8,119,21,156,7,78,5,123,33,80,49,106,23,74,3,90,6,153,7,65,7,91,15,130,17,88,30,106,52,144,12
>super297-68268,168,28,117,16,194,20,210,19,195,22,227,36,118,15,263,28,219,24,220,33,177,25,156,18,217,22,243,43,214,20,155,30,363,53,222,19,322,35,251,23,218,26,203,24,224,15,180,18,133,18,273,19,169,21,92,18,229,24,185,24,208,19,107,16,136,14,193,34,119,11,194,25,198,23,221,41,247,17,187,22
>super2840-258793,56,7,45,0,60,4,65,0,64,1,70,15,33,1,57,3,59,2,65,5,74,3,63,16,50,8,101,2,68,7,42,7,111,3,60,12,68,1,74,6,63,2,76,2,70,6,59,2,47,1,76,4,69,0,32,5,63,4,49,6,70,2,36,2,31,2,58,5,38,1,58,2,60,7,59,2,52,5,64,0


In [32]:
# save, no need to name as 'neutral'
poolsizefile.to_csv(op.join(baydir, 'poolsizefile_HEADERIDX.txt'), sep='\t', index=False)
poolsizefile.to_csv(op.join(baydir, 'poolsizefile_noheaderidx.txt'), sep='\t', index=False, header=False)

#### <center> create commands to run

#### estimate covariance matrix (run in bash)
```bash
cd /lu213/brandon.lind/projects/pool_seq/pangenome/JP_pangenome/JP_pooled/snpsANDindels/03_maf-p05_RD-recalculated/baypass
mkdir neutral_runs
cd neutral_runs

export gfile="/lu213/brandon.lind/projects/pool_seq/pangenome/JP_pangenome/JP_pooled/snpsANDindels/03_maf-p05_RD-recalculated/baypass/neutral_gfile_noheaderidx.txt"

export poolsizefile="/lu213/brandon.lind/projects/pool_seq/pangenome/JP_pangenome/JP_pooled/snpsANDindels/03_maf-p05_RD-recalculated/baypass/poolsizefile_noheaderidx.txt"



# These commands took about 7 hours each to complete


# chain 1
/data/programs/baypass_2.2/sources/g_baypass -gfile $gfile -poolsizefile $poolsizefile \
-nthreads 8 -seed $seed -print_omega_samples -outprefix chain_1 > chain_1_stdout.txt &

# chain 2
/data/programs/baypass_2.2/sources/g_baypass -gfile $gfile -poolsizefile $poolsizefile \
-nthreads 8 -seed $seed -print_omega_samples -outprefix chain_2 > chain_2_stdout.txt &

# chain 3
/data/programs/baypass_2.2/sources/g_baypass -gfile $gfile -poolsizefile $poolsizefile \
-nthreads 8 -seed $seed -print_omega_samples -outprefix chain_3 > chain_3_stdout.txt &

# chain 4
/data/programs/baypass_2.2/sources/g_baypass -gfile $gfile -poolsizefile $poolsizefile \
-nthreads 8 -seed $seed -print_omega_samples -outprefix chain_4 > chain_4_stdout.txt &

# chain 5
/data/programs/baypass_2.2/sources/g_baypass -gfile $gfile -poolsizefile $poolsizefile \
-nthreads 8 -seed $seed -print_omega_samples -outprefix chain_5 > chain_5_stdout.txt &

```